# 1. Logic Definition of Generalization

## (a)

In order to show empirically that n_full/n_avg appraoches 2 for dimensions 2, 4, and 16, I start by creating a random dataset with X being randomly generated by random.rand, and the y labels generated by random.randint with values of 0 or 1 for binary classification. I then, for dimensions of 2, 4, and 8, remove a data point from the dataset and run the knn classifier on the data. In order to determine whether the data point is needed memory, i predict that point on the classifier, and check whether the prediction was correct. If it was, this point was not necessary for fitting the algorithm. We run this 100 times on each dimension and average the count of important points to get n_avg. The output was as follows:

> d=2: n_full=4, Avg. req. points for memorization n_avg=2.22, n_full/n_avg=1.80

> d=4: n_full=16, Avg. req. points for memorization n_avg=8.06, n_full/n_avg=1.99

> d=8: n_full=256, Avg. req. points for memorization n_avg=127.56, n_full/n_avg=2.01

We can see that the values converge to around 2.00.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import random

def generate_random_dataset(dimension, num_points):
    random.seed(1628)
    X = np.random.rand(num_points, dimension)
    y = np.random.randint(2, size=num_points)
    return X, y

def find_important_rows_count(X, y, num_points):
    non_important_rows_count = 0
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(X, y)
    for i in range(num_points):
        X_this = np.delete(X, i, axis=0)
        y_this = np.delete(y, i, axis=0)
        knn.fit(X_this, y_this)
        prediction = knn.predict(X[i].reshape(1, -1))
        if prediction[0] == y[i]:
            non_important_rows_count += 1
    return num_points - non_important_rows_count

dimensions = [2, 4, 8]

for dim in dimensions:
    num_points = 2**dim
    total = 0
    for _ in range(100):
      X, y = generate_random_dataset(dim, num_points)
      count = find_important_rows_count(X, y, num_points)
      total += count
    n_avg = total / 100

    print(f"d={dim}: n_full={num_points}, Avg. req. points for memorization n_avg={n_avg:.2f}, n_full/n_avg={num_points/n_avg:.2f}")

## (b)

I ran my experiment with a 4-class classification problem by generating labels in the range 0 to 3 instead of 0 to 1. The expected results were values approaching c/c-1, in this case 1.33. My outputs did in fact approach 1.33.

> d=2: n_full=4, Avg. req. points for memorization n_avg=2.99, n_full/n_avg=1.34

> d=4: n_full=16, Avg. req. points for memorization n_avg=12.11, n_full/n_avg=1.32

> d=8: n_full=256, Avg. req. points for memorization n_avg=191.96, n_full/n_avg=1.33

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

def generate_random_dataset(dimension, num_points):
    X = np.random.rand(num_points, dimension)
    y = np.random.randint(4, size=num_points)
    return X, y

def find_important_rows_count(X, y, num_points):
    non_important_rows_count = 0
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(X, y)
    for i in range(num_points):
        X_this = np.delete(X, i, axis=0)
        y_this = np.delete(y, i, axis=0)
        knn.fit(X_this, y_this)
        prediction = knn.predict(X[i].reshape(1, -1))
        if prediction[0] == y[i]:
            non_important_rows_count += 1
    return num_points - non_important_rows_count

dimensions = [2, 4, 8]

for dim in dimensions:
    num_points = 2**dim
    total = 0
    for _ in range(100):
      X, y = generate_random_dataset(dim, num_points)
      count = find_important_rows_count(X, y, num_points)
      total += count
    n_avg = total / 100

    print(f"d={dim}: n_full={num_points}, Avg. req. points for memorization n_avg={n_avg:.2f}, n_full/n_avg={num_points/n_avg:.2f}")

# 2. Finite State Machine Generalization:

## (a)

I use the sklearn breast cancer dataset as a binary dataset to run our DT on. I first run the DT and print a tree with export_text, then count the number of if-thens by checking the times when the tree makes a conditional decision or branch. I also note accuracy.

Output:

> if-then count: 30

> original accuracy:  0.9385964912280702

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text

# load data
data = load_breast_cancer()
X = data.data
y = data.target

#fit
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# get feature names as list
feature_names = list(data.feature_names)

# get if-then rules
tree_rules = export_text(clf, feature_names=feature_names)
num_clauses = tree_rules.count("<=") + tree_rules.count(">")

# printing for experiments
accuracy_og = clf.score(X_test, y_test)
print(f"if-then count: {num_clauses}")
print("original accuracy: ", accuracy_og)

**Strategy 1. max_depth**

My first method of reducing if-then clauses was reducing the max_depth of the tree. in order to do this while still maintaining a decent accuracy, I started from 1 and looped up to the depth of the original tree while checking if the proposed depth provides a decent accuracy. For the same run as above, my output was:

> best depth:  1

> if-then count:  2

> accuracy:  0.8947368421052632

> original accuracy:  0.9385964912280702

In [ ]:
## Strategy 1. max_depth ##

og_depth = clf.tree_.max_depth

for i in range(1,og_depth+1):
  clf1 = DecisionTreeClassifier(random_state=42, max_depth=i)
  clf1.fit(X_train, y_train)
  accuracy1 = clf1.score(X_test, y_test)
  if accuracy1 >= accuracy_og-0.2:
    best_depth = i
    break

print("best depth: ", best_depth)
tree_rules_1 = export_text(clf1, feature_names=feature_names)
num_clauses_1 = tree_rules_1.count("<=") + tree_rules_1.count(">")
print("if-then count: ", num_clauses_1)
print("accuracy: ", accuracy1)
print("original accuracy: ", accuracy_og)

**Strategy 2. max_leaf_nodes**

My next method of reducing if-then clauses was reducing the max_leaf_nodes of the tree. This method was very similar to the depth method, just adjusting for max_leaf_nodes instead. For the same run as above, my output was:

> best leaves:  2

> if-then count:  2

> accuracy:  0.8947368421052632

> original accuracy:  0.9385964912280702

In [ ]:
## Strategy 2. leaf nodes ##

og_num_leaf_nodes = clf.tree_.n_leaves

for i in range(2, og_num_leaf_nodes+1):
  clf2 = DecisionTreeClassifier(random_state=42, max_leaf_nodes=i)
  clf2.fit(X_train, y_train)
  accuracy2 = clf2.score(X_test, y_test)
  if accuracy2 >= accuracy_og-0.2:
    best_leaves = i
    break

print("best leaves: ", best_leaves)
tree_rules_2 = export_text(clf2, feature_names=feature_names)
num_clauses_2 = tree_rules_2.count("<=") + tree_rules_2.count(">")
print("if-then count: ", num_clauses_2)
print("accuracy: ", accuracy2)
print("original accuracy: ", accuracy_og)

**Strategy 3. Combination**

Finally, I wanted to try optimizing both the tree depth and leaf node count at the same time. I did this with a nested for-loop as seen below. As a result, I was able to reduce the if-then clauses count to only 2 while maintaining a ~90% accuracy. Notably, combining both methods was equally as effective as either method alone for the same accuracy, as all 3 reduced if-then clauses to 2.

Output:
> final if-then statements count:  2

> final depth:  1

> final leaves:  2

> final accuracy:  0.8947368421052632

> final tree:

|--- mean concave points <= 0.05

|   |--- class: 1

|--- mean concave points >  0.05

|   |--- class: 0

In [ ]:
## Final: combining both metrics ##

for d in range(1,og_depth+1):
  for l in range(2, og_num_leaf_nodes+1):
    clf_final = DecisionTreeClassifier(random_state=42, max_depth=d, max_leaf_nodes=l)
    clf_final.fit(X_train, y_train)
    accuracy_final = clf_final.score(X_test, y_test)
    if accuracy_final >= accuracy_og-0.2:
      final_depth, final_leaves = d, l
      break

# clf_final = DecisionTreeClassifier(random_state=42, max_leaf_nodes=best_leaves, max_depth=best_depth)
# clf_final.fit(X_train, y_train)

tree_rules_final = export_text(clf_final, feature_names=feature_names)
num_clauses_final = tree_rules_final.count("<=") + tree_rules_final.count(">")

print("final if-then statements count: ", num_clauses_final)
print("final depth: ", clf_final.tree_.max_depth)
print("final leaves: ", clf_final.tree_.n_leaves)
print("final accuracy: ", clf_final.score(X_test, y_test))
print()
print("final tree: ")
print(tree_rules_final)

## (b)

Here, I attempt the same method as in (a) on a different sklearn dataset, which is a 10-class classification for digits. This dataset has a much larger initial if-then clause count and lower initial accuracy.

Output:
> if-then count: 40

> original accuracy:  0.8771929824561403

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.metrics import accuracy_score

# load data
data = load_breast_cancer()
X = data.data
y = data.target

#fit
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# get feature names as list
feature_names = list(data.feature_names)

# get if-then rules
tree_rules = export_text(clf, feature_names=feature_names)
num_clauses = tree_rules.count("<=") + tree_rules.count(">")

y_pred = clf.predict(X_test)
accuracy_og = accuracy_score(y_test, y_pred)

# printing for experiments
#accuracy_og = clf.score(X_test, y_test)
print(f"if-then count: {num_clauses}")
print("original accuracy: ", accuracy_og)

**If-then reduction method: Combined**

Because the method of combining depth analysis and leaf node analysis resulted in the same number of if-then clauses as the separate methods and with similar accuracy, I opted to simply run the combined script. The results were very similar to my original dataset, but with a slightly lower accuracy of 88.5%.

> final if-then statements count:  2

> final depth:  1

> final leaves:  2

> final accuracy:  0.8859649122807017

> final tree:

|--- worst perimeter <= 113.75

|   |--- class: 1

|--- worst perimeter >  113.75

|   |--- class: 0


In [ ]:
## Final: finding best depth and num. leaf nodes ##
og_depth = clf.tree_.max_depth
og_num_leaf_nodes = clf.tree_.n_leaves

for d in range(1,og_depth+1):
  for l in range(2, og_num_leaf_nodes+1):
    clf_final = DecisionTreeClassifier(random_state=45, max_depth=d, max_leaf_nodes=l)
    clf_final.fit(X_train, y_train)
    accuracy_final = clf_final.score(X_test, y_test)
    if accuracy_final >= accuracy_og-0.2:
      final_depth, final_leaves = d, l
      break

tree_rules_final = export_text(clf_final, feature_names=feature_names)
num_clauses_final = tree_rules_final.count("<=") + tree_rules_final.count(">")

print("final if-then statements count: ", num_clauses_final)
print("final depth: ", clf_final.tree_.max_depth)
print("final leaves: ", clf_final.tree_.n_leaves)
print("final accuracy: ", accuracy_final)
print()
print("final tree: ")
print(tree_rules_final)

## (c)

To generate the random dataset for use in this section, I used the same method as in 6.1 with binary classification. Due to the nature of random datasets, the accuracy was lower than the other attempts, as well as the number of if-then clauses.

> if-then count: 64
> original accuracy:  0.65

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import export_text
from sklearn.metrics import accuracy_score

def generate_random_dataset(dimension, num_points):
    X = np.random.rand(num_points, dimension)
    y = np.random.randint(2, size=num_points)
    return X, y

X, y = generate_random_dataset(2, 100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

clfrand = DecisionTreeClassifier()
clfrand.fit(X_train, y_train)

tree_rules = export_text(clfrand)
num_clauses = tree_rules.count("<=") + tree_rules.count(">")

y_pred = clfrand.predict(X_test)
accuracy_og = accuracy_score(y_test, y_pred)

print(f"if-then count: {num_clauses}")
print("original accuracy: ", accuracy_og)

Because the accuracy is much more volatile for random datasets, I opted to only select a depth-leaf-node combination if it equaled or exceeded our original accuracy for this run. This lead to a final if-then count of 2 and an increase in accuracy to 0.7.

> final if-then statements count:  2

> final depth:  1

> final leaves:  2

> final accuracy:  0.7

> final tree:

|--- feature_0 <= 0.89

|   |--- class: 1

|--- feature_0 >  0.89

|   |--- class: 0

In [ ]:
## Final: finding best depth and num. leaf nodes ##
og_depth = clfrand.tree_.max_depth
og_num_leaf_nodes = clfrand.tree_.n_leaves

for d in range(1,og_depth+1):
  for l in range(2, og_num_leaf_nodes+1):
    clf_final = DecisionTreeClassifier(random_state=45, max_depth=d, max_leaf_nodes=l)
    clf_final.fit(X_train, y_train)
    y_pred = clf_final.predict(X_test)
    accuracy_final = accuracy_score(y_test, y_pred)
    if accuracy_final >= accuracy_og:
      final_depth, final_leaves = d, l
      break

tree_rules_final = export_text(clf_final)
num_clauses_final = tree_rules_final.count("<=") + tree_rules_final.count(">")

print("final if-then statements count: ", num_clauses_final)
print("final depth: ", clf_final.tree_.max_depth)
print("final leaves: ", clf_final.tree_.n_leaves)
print("final accuracy: ", accuracy_final)
print()
print("final tree: ")
print(tree_rules_final)

# 3. Compression

## (a)

I used the zlib.compress function to compress a randomly generated string of size 200, arbitrarily selected. The results were as follows:

> Original string length: 200

> Compressed string length: 199

> Compression ratio: .995

In [ ]:
import random
import string
import zlib

#generate random string
chars = string.ascii_letters + string.digits + string.punctuation
strand = ''.join(random.choice(chars) for _ in range(200))

print(strand)

#apply compression
encoded = strand.encode()
compressed = zlib.compress(strand.encode())

#calculate ratio
ratio = len(compressed) / len(strand)

#print results
print("Original string length:", len(strand))
print("Compressed string length:", len(compressed))
print("Compression ratio:", ratio)

N>kt|6e;k)AM2~@B)9IrJ.;;NG#0gzIHf~EgBTh?\E9IxI3e$,O-}c1*KjO*.opPc+X:rLv{AKeAQhC?1,*F*~CZ!Fn5Ma$n556jGYd-]gHze`jXG@hagN15|Li&;S+kFd9bd(P93hHXZ7ptV72!`OqZli~a.jb\@4H!O!`dQlb"A@U5_CV149fXn{p9OoU^$H]%Ek<"
Original string length: 200
Compressed string length: 199
Compression ratio: 0.995


## (b)

The expected compression ratio in (a) is approximately 1. This is because when viewed through its relationship to generalization, we expect that when G is close to 1, this indicates no compression and the model hasn't reduced complexity. This is expected for a randomly generated string, as it is highly generalized so there is minimal compression.